定义网络

In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch as t
from torch.autograd import Variable

class Net(nn.Module):
    def __init__(self):
        #nn.Module子类的函数必须再构造函数种执行父类的构造函数
        #下式等价于nn.Module.__init__(self)
        super(Net,self).__init__()
        #卷积层'1'表示输入图片为单通道，‘6’表示输出通道数
        #‘5’表示卷积核为5*5
        self.conv1 = nn.Conv2d(1,6,5)
        #卷积层
        self.conv2 = nn.Conv2d(6,16,5)
        #仿射层/全连接层，y=Wx+b
        self.fc1=nn.Linear(16*5*5,120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)
        
    def forward(self,x):
        #卷积 --> 激活 --> 池化
        x=F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x=F.max_pool2d(F.relu(self.conv2(x)),2)
        #reshape, '-1'表示自适应
        x=x.view(x.size()[0],-1)
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [2]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


In [31]:
for name, parameters in net.named_parameters():
    print(name,':',parameters.size())

conv1.weight : torch.Size([6, 1, 5, 5])
conv1.bias : torch.Size([6])
conv2.weight : torch.Size([16, 6, 5, 5])
conv2.bias : torch.Size([16])
fc1.weight : torch.Size([120, 400])
fc1.bias : torch.Size([120])
fc2.weight : torch.Size([84, 120])
fc2.bias : torch.Size([84])
fc3.weight : torch.Size([10, 84])
fc3.bias : torch.Size([10])


In [32]:

input = Variable(t.randn(1, 1, 32, 32))
out=net(input)
out.size()

torch.Size([1, 10])

In [33]:
net.zero_grad() #所有参数的梯度清零
out.backward(Variable(t.ones(1,10))) #反向传播

损失函数

In [44]:
output = net(input)
target = Variable(t.ones(1,10)) #这里原来是target = Variable(t.arange(0,10))现实类型错误，暂时不知道为啥
print(target.type)
criterion = nn.MSELoss()
loss = criterion(output, target)
loss

<built-in method type of Tensor object at 0x0000028ED625E480>


tensor(0.9390, grad_fn=<MseLossBackward>)

input  -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
     -> view -> linear -> relu -> linear -> relu -> linear
     -> MSELoss
     -> loss

In [46]:
# 运行 .backward，观察调用之前和调用之后的grad
net.zero_grad() # 把net中所有可学习参数的梯度清零
print('反向传播之前 conv1.bias 的梯度')
print(net.conv1.bias.grad)
loss.backward()
print('反向传播之后 conv1.bias的梯度')
print(net.conv1.bias.grad)

反向传播之前 conv1.bias 的梯度
tensor([0., 0., 0., 0., 0., 0.])
反向传播之后 conv1.bias的梯度
tensor([-0.0022, -0.0130, -0.0005, -0.0061, -0.0016, -0.0123])


优化器

随机梯度下降法（SGD）的更新策略：
weight=weight-learning_rate*gradinet

In [48]:
learning_rate=0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)#inplace减法

In [ ]:
import torch.optim as optim
optimizer = optim.SDG(net.parameters(),lr=0.01)

#在训练过程中
#先梯度清零（与net.zero_grad()效果一样）
optimizer.zero_grad()

#计算损失
output=net(input)
loss=criterion(output, target)

#反向传播
loss.backward()

#更新参数
optimizer